# Sentimental Analysis

<br>

#### 군산대학교 한국어 감성사전 사용
https://github.com/park1200656/KnuSentiLex

In [7]:
sentiment_dict = {}
path = './sentiment-dict/'

with open(path+'pos_pol_word.txt', 'r') as f:
    sentiment_dict['pos'] = f.read().split("\n")[19:]
    
with open(path+'neg_pol_word.txt', 'r') as f:
    sentiment_dict['neg'] = f.read().split("\n")[19:]

In [10]:
sentiment_dict['pos'][:5]

['(-;', '(^^)', '(^-^)', '(^^*', '(^_^)']

In [11]:
sentiment_dict['neg'][:5]

['가난', '가난뱅이', '가난살이', '가난살이하다', '가난설음']

In [54]:
def get_sentiment(tokens):
    pos, neg = 0, 0    
    for token in tokens:
        if token in sentiment_dict['pos']:
            pos += 1
        if token in sentiment_dict['neg']:
            neg += 1
    return [0, 1][pos>neg]

## Naver News 감성분석 실습

In [38]:
import requests
from bs4 import BeautifulSoup
import re
from konlpy.tag import Mecab
import kss

In [39]:
def get_text_from_naver_news(url):
    #url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108'
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    news_with_tags = soup.select('div._article_body_contents')[0]
    
    sentences = []
    for splist in sentence_split:
        for sentence in splist:
            if '. ' in sentence:
                sentences += sentence.split('. ')
            else:
                sentences.append(sentence)
                
    sentences = [re.sub('<.+>', ' ', s) for s in sentences
                if re.sub('<.+>', ' ', s) not in ['', ' ', '  ']]
    
    email_idx = 0
    email_ptn = '[a-zA-Z0-9_\-.]+@[a-z]+(?:\.[a-z]+)+'
    sentences_raw = []

    for sentence in sentences:
        if '오류를 우회하기 위한' not in sentence:
            if re.findall(email_ptn, sentence):
                break
            sentences_raw.append(sentence)
        else:
            pass
        
    return sentences_raw

In [52]:
url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=015&aid=0004513867'
sentences = get_text_from_naver_news(url)
sentences

['주호영 "LH 특검 수용, 여야 국회의원 전수조사 제안"',
 '민주당 "주호영 요구 즉각 수용, 전수조사 바로 진행"',
 '주호영 국민의힘 원내대표가 16일 긴급 기자회견을 열고 더불어민주당이 제안한 한국토지주택공사(LH) 사태 특검 수사 제안을 수용하겠다고 밝혔다',
 '앞서 국민의힘은 특검 출범까지 약 한 달이 소요되기 때문에 그동안 증거인멸이 우려된다며 특검 제안을 거부했다',
 '국민의힘은 일단 검찰이 수사에 착수한 후 특검이 출범하면 그동안의 수사내용을 이첩하자는 입장이었다.',
 '이날 주호영 원내대표는 국회 소통관에서 기자회견을 갖고 특검 뿐만 아니라 국회의원 전원과 직계존비속, 지방공적체는 물론 청와대 전수조사까지 역제안했다',
 '주호영 원내대표는 "오늘 민주당의 한 의원이 3기 신도시 투기 의혹이 불거진 자신의 땅에 갑자기 감자를 심었다는 기사가 나왔다',
 '대통령부터 민주당 의원들까지 문재인 정권 인사 중 영농인을 자처하는 분들이 급증하면서 국민의 분노가 하늘을 찌른다"며 "국민의힘은 거두절미하고 국회의원 대상 강력한 전수조사는 물론 특검과 국정조사 실시를 요구한다"고 했다',
 '이어 "이번 3월 회기 중에 LH특검법안이 본회의에서 즉시 처리되도록 특검법 공동발의에 민주당은 즉각 협조하라',
 '특검이 실시될 때까지, 한두 달의 시간 동안 범죄자들이 증거인멸 여지를 갖지 못하도록 수사를 전담한 국수본은 조직의 명운을 걸고 강도 높은 수사를 벌여라',
 '검경수사권 조정의 공백 우려를 이번 LH투기 범죄 수사를 계기로 불식시키기 바란다"고 요구했다.',
 ' 9일 경기남부경찰청 반부패경제범죄수사대가 신도시 투기 의혹과 관련해 압수수색중인 경기도 광명시 일직동 한국토지주택공사(LH) 광명시흥사업본부에 적막감이 감돌고 있다',
 '사진=뉴스1 ',
 '그는 "국정조사요구서는 빠르면 오늘 중 제출하겠다',
 '항간에는 이번 3기 신도시 LH투기 파문과 관련해 공급의 명분과 개발이익의 극대화를 노리는 거대세력이 전국의 부동산값을 천정부지로

In [53]:
[get_sentiment(s) for s in sentences]

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]

In [55]:
sum([get_sentiment(s) for s in sentences]) / len([get_sentiment(s) for s in sentences])

0.14285714285714285

In [20]:
url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108'
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
news_with_tags = soup.select('div._article_body_contents')[0]
kss.split_sentences(news_with_tags.text)

['// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}\n\n주호영 "LH 특검 수용, 여야 국회의원 전수조사 제안"민주당 "주호영 요구 즉각 수용, 전수조사 바로 진행"국민의힘 주호영 원내대표. 사진=연합뉴스주호영 국민의힘 원내대표가 16일 긴급 기자회견을 열고 더불어민주당이 제안한 한국토지주택공사(LH) 사태 특검 수사 제안을 수용하겠다고 밝혔다.',
 '앞서 국민의힘은 특검 출범까지 약 한 달이 소요되기 때문에 그동안 증거인멸이 우려된다며 특검 제안을 거부했다.',
 '국민의힘은 일단 검찰이 수사에 착수한 후 특검이 출범하면 그동안의 수사내용을 이첩하자는 입장이었다.',
 '이날 주호영 원내대표는 국회 소통관에서 기자회견을 갖고 특검 뿐만 아니라 국회의원 전원과 직계존비속, 지방공적체는 물론 청와대 전수조사까지 역제안했다.',
 '주호영 원내대표는 "오늘 민주당의 한 의원이 3기 신도시 투기 의혹이 불거진 자신의 땅에 갑자기 감자를 심었다는 기사가 나왔다. 대통령부터 민주당 의원들까지 문재인 정권 인사 중 영농인을 자처하는 분들이 급증하면서 국민의 분노가 하늘을 찌른다"며 "국민의힘은 거두절미하고 국회의원 대상 강력한 전수조사는 물론 특검과 국정조사 실시를 요구한다"고 했다.',
 '이어 "이번 3월 회기 중에 LH특검법안이 본회의에서 즉시 처리되도록 특검법 공동발의에 민주당은 즉각 협조하라. 특검이 실시될 때까지, 한두 달의 시간 동안 범죄자들이 증거인멸 여지를 갖지 못하도록 수사를 전담한 국수본은 조직의 명운을 걸고 강도 높은 수사를 벌여라. 검경수사권 조정의 공백 우려를 이번 LH투기 범죄 수사를 계기로 불식시키기 바란다"고 요구했다.',
 '9일 경기남부경찰청 반부패경제범죄수사대가 신도시 투기 의혹과 관련해 압수수색중인 경기도 광명시 일직동 한국토지주택공사(LH) 광명시흥사업본부에 적막감이 감돌고 있다.',
 '사진=뉴스1그는 "국정조사요구서는 빠르면 오늘 중 제출하겠다. 항간에는 이번

In [56]:
[get_sentiment(k) for k in kss.split_sentences(news_with_tags.text)]

[0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0]

## Naive Bayes 감정 분류

In [42]:
train = [("I love you", 1),
        ("love you happy weekend", 1),
        ("bore work job", 0),
        ("I hate you", 0),
        ("bore weekend", 0),
        ("happy together", 1)]

In [58]:
from collections import defaultdict
import numpy as np
    
def detect_spam_NB(training_set, email_words, unseen_word_accept=True):
    
    doc_cnt0 = 0 
    doc_cnt1 = 0 

    wordfreq = defaultdict(lambda : [0, 0])

    for doc, label in training_set:
        words = doc.split()
        for word in words:
            wordfreq[word][label] += 1
    
    # word check
    for word in email_words:
        if word not in wordfreq.keys():
            if not unseen_word_accept:
                assert word not in email_words, "Invalid word list." # assert
            wordfreq[word] = [1, 1] # set 50% / 50%
                
    
    for key, (cnt0, cnt1) in wordfreq.items():
        doc_cnt0 += cnt0 # normal count
        doc_cnt1 += cnt1 # spam count
    
    cond_prob_dict = wordfreq.copy()
    k = 0.5
    prob_norm = (doc_cnt0) / (doc_cnt0 + doc_cnt1)
    prob_spam = 1 - prob_norm
    
    # Laplace Smoothing 적용
    for key, (cnt0, cnt1) in wordfreq.items():
        # 스팸메일일 때 해당 단어가 출현할 확률
        prob_word_given_1 = (k + cnt1) / (2 * k + doc_cnt1)

        # 일반메일일 때 해당 단어가 출현할 확률
        prob_word_given_0 = (k + cnt0) / (2 * k + doc_cnt0)
        
        # cond_prob_dict 업데이트
        cond_prob_dict[key] = [prob_word_given_0, prob_word_given_1]
    
    # ㅣog 변환 - 언더플로우 방지
    log_cond_prob_dict = {key:[np.log(v0), np.log(v1)] for (key,[v0, v1]) in cond_prob_dict.items()}
    
    #print(log_cond_prob_dict)
    
    # 계산
    numerators_0_given_words = np.exp(np.sum([log_cond_prob_dict[word][0] for word in email_words]))
    numerators_1_given_words = np.exp(np.sum([log_cond_prob_dict[word][1] for word in email_words]))
    denominator = numerators_0_given_words + numerators_1_given_words
    
    #print(numerators_spam_given_words, denominator)
    
    prob_0_given_words = numerators_0_given_words / denominator
    prob_1_given_words = 1 - prob_0_given_words
    
    return round(prob_1_given_words, 3)

In [59]:
detect_spam_NB(train, ["love", "weekend"])

0.802

In [67]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.model_selection import GridSearchCV

In [ ]:
text_clf = Pipeline([('vect', CountVectorizer()), # DTM - Document-Term Matrix
                     ('tfidf', TfidfTransformer()), # tf-idf 계산
                     ('clf', MultinomialNB())]) # tf-idf 수치 기반으로 NB classification

text_clf = text_clf.fit(twenty_train.data,
                        twenty_train.target)

In [ ]:
twenty_test = fetch_20newsgroups(subset='test', shuffle=True)
predicted = text_clf.predict(twenty_test.data)

# accuracy
np.mean(predicted == twenty_test.target)

In [ ]:
parameters = {'vect__ngram_range' : [(1, 1), (1, 2)], # (1,1) = unigram, (1,2) = bigram
              'tfidf__use_idf' : (True, False),
              'clf__alpha' : (0.001, 0.01, 0.1, 0.5, 1.0)
             }

gscv_clf = GridSearchCV(text_clf, parameters,
                       n_jobs=-1, verbose=3)
gscv_clf_fit = gscv_clf.fit(twenty_train.data, twenty_train.target)

In [ ]:
gscv_clf.best_score_

In [ ]:
predicted = gscv_clf.best_estimator_.predict(twenty_test.data)
np.mean(predicted == twenty_test.target)

# ratings.txt 분석
using Scikit-learn

In [86]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV

In [79]:
df = pd.read_csv(path+'ratings.txt', sep='\t').dropna()

In [80]:
df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [85]:
df.shape

(199992, 3)

In [81]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['document'],
                                                   df['label'],
                                                   random_state=44)

In [87]:
text_clf = Pipeline([('vect', CountVectorizer()), # DTM - Document-Term Matrix
                     ('tfidf', TfidfTransformer()), # tf-idf 계산
                     ('clf', MultinomialNB())]) # tf-idf 수치 기반으로 NB classification

text_clf = text_clf.fit(X_train, y_train)

In [88]:
predicted = text_clf.predict(X_test)

# accuracy
np.mean(predicted == y_test)

0.8332533301332053

In [89]:
parameters = {'vect__ngram_range' : [(1, 1), (1, 2)], # (1,1) = unigram, (1,2) = bigram
              'tfidf__use_idf' : (True, False),
              'clf__alpha' : (0.001, 0.01, 0.1, 0.5, 1.0)
             }

gscv_clf = GridSearchCV(text_clf, parameters,
                       n_jobs=-1, verbose=3)
gscv_clf_fit = gscv_clf.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   50.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  3.2min finished


In [90]:
gscv_clf_pred = gscv_clf_fit.best_estimator_.predict(X_test)
np.mean(gscv_clf_pred == y_test)

0.8370534821392855

In [92]:
pd.DataFrame({'review' : X_test,
              'predict' : gscv_clf_pred,
              'real' : y_test})

,review,predict,real
92366,왜 요즘에는 이런영화가 나오지 않는 것인가! 광해? 7번방의선물? 신세계? 개나줘버려!,1,1
172729,샴푸의 Trouble만 좋았음,1,0
495,완전 짱!!!,1,1
158066,"진정성이 느껴지지 않는, 그저그런 영화.",0,0
8246,권상우씨 풋풋했을때 정말 멋있으셨네^^ 그리고종혁아빠는 보다가 종혁아빠아닌줄 ㄷㄷ ...,1,1
...,...,...,...
193820,제목을 액션임파서블 아니면 무술임파서블 이라 하면 좋을 듯~! 완벽한 톰형 최대의....,0,0
136388,지루하다. 어색한 사투리,0,0
120008,너무 황당해서 로그인하게 만드는 영화네. 설마 죽을줄은 ㅡㅡ??,0,0
75562,포켓몬 받으려구....,0,1
